#Asignatura: Visión por Computador

##Alumnos: Yeray Álvarez-Buylla Parra, María Elena Navarro Santana

##Práctica 4

###Paquetes necesarios

In [5]:
import math 

from ultralytics import YOLO

import csv
import cv2
from ultralytics import YOLO
from collections import defaultdict, deque
import easyocr
import time

TAREA: desarrollar un prototipo que procese uno (vídeo ejemplo proporcionado) o varios vídeos (incluyendo vídeos de cosecha propia):
- detecte y siga las personas y vehículos presentes
- detecte y lea las matrículas de los vehículos presentes
- cuente el total de cada clase
- vuelque a disco un vídeo que visualice los resultados
- genere un archivo csv con el resultado de la detección y seguimiento. Se sugiere un formato con al menos los siguientes campos:

fotograma, tipo_objeto, confianza, identificador_tracking, x1, y1, x2, y2, matrícula_en_su_caso, confianza, mx1,my1,mx2,my2, texto_matricula


In [2]:
# Etiquetas de las clases
classNames = ["matricula", "person", "car", "motorbike"]

# Carga el modelo YOLO y el modelo entrenado de 100 épocas
modelo_general = YOLO('yolo11n.pt')  
modelo_matriculas = YOLO('best.pt')   

# Inicialización del contador para cada clase
contador_clases = defaultdict(int)

# Archivo de video a analizar
filename = "video_2.mp4"

# Inicialización del lector OCR
lector_ocr = easyocr.Reader(['es'], gpu=False)

# Lista de matrículas detectadas y tiempo de detección
matriculas_detectadas = deque(maxlen=10)  # Mantiene un máximo de 10 matrículas visibles
tiempo_visibilidad = 5  # Tiempo en segundos que se mantendrá visible una matrícula detectada

# Nombre del archivo CSV
csv_filename = "detecciones.csv"

# Crea el archivo CSV y escribe el encabezado
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([
        "fotograma", "tipo_objeto", "confianza", "identificador_tracking",
        "x1", "y1", "x2", "y2", "matricula_en_su_caso", "confianza_matricula",
        "mx1", "my1", "mx2", "my2", "texto_matricula"
    ])

# Configuración para guardar un video con las detecciones
cap = cv2.VideoCapture(filename)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter('video_2_detecciones.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

# Función para validar matrículas
def es_matricula_valida(texto):
    texto = texto.replace(" ", "")
    return len(texto) == 7 and texto[:4].isdigit() and texto[4:].isalpha()

# Función para procesar detecciones de vehículos y personas
def procesar_detecciones_generales(results, frame, fotograma_num):
    vehiculos = []
    posiciones_vehiculos = []
    
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = round(box.conf[0].item(), 2)
            class_id = int(box.cls[0])  # Índice de clase del objeto detectado
            
            # Asegurarse de que solo se procesen personas, coches y motos en yolo11n.pt
            if class_id < len(classNames) and classNames[class_id] in ["person", "car", "motorbike"]:
                tipo_objeto = classNames[class_id]  # Nombre de la clase detectada
                
                # Definir color según el tipo de objeto
                color = (0, 0, 255) if tipo_objeto == "person" else (255, 0, 0)
                
                # Dibujar el rectángulo y etiqueta en el frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 3)
                cv2.putText(frame, f"{tipo_objeto} {confidence}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                
                # Registrar solo coches y motos para OCR y detección de matrículas
                if tipo_objeto in ["car", "motorbike"]:
                    vehiculos.append(frame[y1:y2, x1:x2])  # Extraer el vehículo detectado
                    posiciones_vehiculos.append((x1, y1))  # Guardar la posición para cálculo global

                # Escribir en el CSV la detección de cada objeto
                with open(csv_filename, mode='a', newline='') as file:
                    writer = csv.writer(file)
                    writer.writerow([
                        fotograma_num, tipo_objeto, confidence, class_id,
                        x1, y1, x2, y2, "", "", "", "", "", ""
                    ])
    
    return vehiculos, posiciones_vehiculos

# Función para procesar detecciones de matrículas dentro de los vehículos
def procesar_detecciones_matriculas(vehiculos, posiciones_vehiculos, frame, fotograma_num):
    for idx, vehiculo in enumerate(vehiculos):
        results = modelo_matriculas(vehiculo)
        
        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = round(box.conf[0].item(), 2)
                
                if confidence > 0.5:
                    # Se calcula la posición global de la matrícula
                    x1_global = posiciones_vehiculos[idx][0] + x1
                    y1_global = posiciones_vehiculos[idx][1] + y1
                    x2_global = posiciones_vehiculos[idx][0] + x2
                    y2_global = posiciones_vehiculos[idx][1] + y2
                    
                    # Dibuja el recuadro y etiqueta de matrícula
                    cv2.rectangle(frame, (x1_global, y1_global), (x2_global, y2_global), (0, 255, 0), 3)
                    cv2.putText(frame, f"matricula {confidence}", (x1_global, y1_global - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

                    # Extrae y procesa OCR en la imagen de la matrícula
                    placa_imagen = frame[y1_global:y2_global, x1_global:x2_global]
                    resultado_ocr = lector_ocr.readtext(placa_imagen)
                    
                    texto_matricula = ''
                    if resultado_ocr:
                        textos_detectados = [item[1] for item in resultado_ocr]
                        texto_matricula = ' '.join(textos_detectados)
                        matriculas_detectadas.append((texto_matricula, time.time()))
                    
                    # Escribe en el CSV la detección de la matrícula
                    with open(csv_filename, mode='a', newline='') as file:
                        writer = csv.writer(file)
                        writer.writerow([
                            fotograma_num, "matricula", confidence, idx,
                            x1_global, y1_global, x2_global, y2_global,
                            "matricula", confidence,
                            x1, y1, x2, y2, texto_matricula
                        ])

# Función para mostrar matrículas en pantalla
def mostrar_matriculas_en_pantalla(frame):
    matriculas_a_mostrar = [mat for mat, t in matriculas_detectadas if time.time() - t < tiempo_visibilidad]
    
    for idx, matricula in enumerate(matriculas_a_mostrar):
        cv2.putText(frame, f'Matricula: {matricula}', (10, 30 + idx * 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

# Procesa el video con detección en tiempo real
fotograma_num = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    fotograma_num += 1
    results_generales = modelo_general(frame)
    vehiculos, posiciones_vehiculos = procesar_detecciones_generales(results_generales, frame, fotograma_num)
    
    if vehiculos:
        procesar_detecciones_matriculas(vehiculos, posiciones_vehiculos, frame, fotograma_num)
    
    mostrar_matriculas_en_pantalla(frame)
    cv2.imshow('Detector de Matriculas', frame)
    
    # Escribe el frame con las detecciones en el video de salida
    out.write(frame)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Libera los recursos
cap.release()
out.release()
cv2.destroyAllWindows()


Using CPU. Note: This module is much faster with a GPU.



0: 640x384 1 car, 1 motorcycle, 1 stop sign, 149.5ms
Speed: 1.9ms preprocess, 149.5ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 384)

0: 416x352 (no detections), 97.4ms
Speed: 1.6ms preprocess, 97.4ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 352)

0: 416x416 (no detections), 108.7ms
Speed: 1.0ms preprocess, 108.7ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 640x384 1 car, 1 motorcycle, 1 stop sign, 91.0ms
Speed: 2.4ms preprocess, 91.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 416x384 (no detections), 101.7ms
Speed: 1.3ms preprocess, 101.7ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 384)

0: 416x416 (no detections), 78.5ms
Speed: 1.0ms preprocess, 78.5ms inference, 1.0ms postprocess per image at shape (1, 3, 416, 416)

0: 640x384 1 car, 1 motorcycle, 1 stop sign, 94.3ms
Speed: 2.0ms preprocess, 94.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 384)

0: 416x384 